In [42]:
import pandas as pd
import numpy as np
from datetime import datetime
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import ta
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
df = pd.read_csv("cnbc_headlines.csv")

month_mapping = {
    "Jan": "January",
    "Feb": "February",
    "Aug": "August",
    "Sept": "September",
    "Oct": "October",
    "Nov": "November",
    "Dec": "December",
    "Jul":"July",
    "Jun":"June",
    "Mar":"March",
    "Apr":"April"
}

# Replace abbreviated month names with full names using str.replace and a loop
for abbrev, full in month_mapping.items():
    df["Time"] = df["Time"].str.replace(abbrev, full)

def extract_date(time_str):
    if pd.isna(time_str):
        return None
    try:
        # The date starts after "ET" or "ET Fri,"
        parts = time_str.split(",")[-1].strip()
        date = pd.to_datetime(parts, format='%d %B %Y', errors='coerce')  # Parse the date
        return date
    except Exception as e:
        return None

df['Date'] = df['Time'].apply(extract_date)
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)
df_cnbc = df.copy()

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
df=pd.read_csv("guardian_headlines.csv")
for abbrev, full in month_mapping.items():
    df["Time"] = df["Time"].str.replace(abbrev, full)
df["Date"] = pd.to_datetime(df["Time"], errors="coerce")
df = df[df["Date"]<datetime(2024,12,25)]
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)
df_guardian = df.copy()
df_guardian

/var/folders/pk/90dn4p5556l45snxl195_hlm0000gn/T/ipykernel_38426/3753698641.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Time"], errors="coerce")


,Time,Headlines,Date
0,18-July-20,Johnson is asking Santa for a Christmas recovery,2020-07-18
1,18-July-20,‘I now fear the worst’: four grim tales of wor...,2020-07-18
2,18-July-20,Five key areas Sunak must tackle to serve up e...,2020-07-18
3,18-July-20,Covid-19 leaves firms ‘fatally ill-prepared’ f...,2020-07-18
4,18-July-20,The Week in Patriarchy \n\n\n Bacardi's 'lad...,2020-07-18
...,...,...,...
17755,17-December-17,How investing in solar energy can create a bri...,2017-12-17
17756,17-December-17,Poundland suppliers hit by insurance downgrade,2017-12-17
17757,17-December-17,Cryptocurrencies: City watchdog to investigate...,2017-12-17
17758,17-December-17,Unilever sells household name spreads to KKR f...,2017-12-17


In [3]:
df = pd.read_csv("reuters_headlines.csv")
for abbrev, full in month_mapping.items():
    df["Time"] = df["Time"].str.replace(abbrev, full)
df["Date"] = pd.to_datetime(df["Time"], errors="coerce")
df = df[df["Date"]<datetime(2024,12,25)]
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)
df_reuters = df.copy()

In [4]:
df_main = pd.concat([df_cnbc, df_reuters, df_guardian], ignore_index=True)
df_main = df_main.loc[:,["Headlines","Date"]]


In [5]:
tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
from nltk.corpus import stopwords
import nltk

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    words = str(text).split()
    # Remove stopwords and join the first 512 characters
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)[:512]

# Apply cleaning to the content
df_main['Headlines'] = df_main['Headlines'].apply(clean_text)

tqdm.pandas()
df_main['sentiment_finbert'] = df_main['Headlines'].progress_apply(lambda x: finbert(x)[0])

Device set to use mps:0
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhinavchhabra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 50253/50253 [18:44<00:00, 44.67it/s]


In [ ]:
df_main['sentiment_label_finbert'] = df_main['sentiment_finbert'].apply(lambda x: x['label'].lower())
df_main['sentiment_score_finbert'] = df_main['sentiment_finbert'].apply(lambda x: x['score'])
sentiment_mapping = {"neutral": 0, "positive": 1, "negative": -1}
df_main['sentiment_numeric_finbert'] = df_main['sentiment_label_finbert'].map(sentiment_mapping)
df_main["sentiment_score_finbert"] = df_main["sentiment_numeric_finbert"]*df_main["sentiment_score_finbert"]
df_main = df_main.loc[:,["Date","sentiment_numeric_finbert","sentiment_score_finbert"]]
df_main

,Date,sentiment_numeric_finbert,sentiment_score_finbert
0,2020-05-29,0,0.000000
1,2020-05-29,1,0.663114
2,2020-05-29,0,0.000000
3,2020-05-29,1,0.999976
4,2020-05-29,1,0.974604
...,...,...,...
50248,2017-12-17,1,1.000000
50249,2017-12-17,-1,-0.999068
50250,2017-12-17,0,0.000000
50251,2017-12-17,0,0.000000


In [7]:
df_main["Date"].max()

Timestamp('2020-07-18 00:00:00')

In [8]:
import yfinance as yf
ticker = "^GSPC" 
start_date = "2017-12-17"
end_date = "2020-07-18"
sp500_data = yf.download(ticker, start=start_date, end=end_date)
sp500_data.reset_index(inplace=True)
sp500_data

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
0,2017-12-18,2690.159912,2694.969971,2685.919922,2685.919922,3727770000
1,2017-12-19,2681.469971,2694.439941,2680.739990,2692.709961,3407680000
2,2017-12-20,2679.250000,2691.010010,2676.110107,2688.179932,3246230000
3,2017-12-21,2684.570068,2692.639893,2682.399902,2683.020020,3293130000
4,2017-12-22,2683.340088,2685.350098,2678.129883,2684.219971,2401030000
...,...,...,...,...,...,...
644,2020-07-13,3155.219971,3235.320068,3149.429932,3205.080078,4902440000
645,2020-07-14,3197.520020,3200.949951,3127.659912,3141.110107,4507140000
646,2020-07-15,3226.560059,3238.280029,3200.760010,3225.979980,4686830000


In [14]:
sp500_data.columns = ['Date','Close',(  'High', '^GSPC'),
            (   'Low', '^GSPC'),
            (  'Open', '^GSPC'),
            ('Volume', '^GSPC')]

In [16]:
merged_df = pd.merge(df_main,sp500_data,on="Date",how="left")
for index, row in merged_df.iterrows():
    if pd.isna(row['Close']):
        next_date = sp500_data[sp500_data['Date'] > row['Date']]['Date'].min()
        merged_df.loc[index, 'Date'] = next_date

final_df = pd.merge(merged_df[["Date","sentiment_numeric_finbert","sentiment_score_finbert"]], sp500_data, on='Date', how='left')
final_df.dropna(inplace=True)
final_df.reset_index(inplace=True,drop=True)
final_df = final_df.loc[:,["Date","sentiment_numeric_finbert","sentiment_score_finbert","Close"]]
final_df

,Date,sentiment_numeric_finbert,sentiment_score_finbert,Close
0,2020-05-29,0,0.000000,3044.310059
1,2020-05-29,1,0.663114,3044.310059
2,2020-05-29,0,0.000000,3044.310059
3,2020-05-29,1,0.999976,3044.310059
4,2020-05-29,1,0.974604,3044.310059
...,...,...,...,...
50224,2017-12-18,1,1.000000,2690.159912
50225,2017-12-18,-1,-0.999068,2690.159912
50226,2017-12-18,0,0.000000,2690.159912
50227,2017-12-18,0,0.000000,2690.159912


In [17]:
sentiment_counts = final_df.groupby(['Date', 'sentiment_numeric_finbert']).size().unstack(fill_value=0)
sentiment_counts.reset_index(inplace=True)
sentiment_counts.columns =["Date", "Count_-1","Count_0","Count_1"]
grouped_df = final_df.groupby(by="Date").mean()
grouped_df.reset_index(inplace=True)
grouped_df.sort_values(by="Date", inplace=True)
grouped_df = pd.merge(grouped_df, sentiment_counts, on="Date", how="left")

# Adding Count Ratios
total_counts = np.sum([grouped_df["Count_-1"], grouped_df["Count_0"], grouped_df["Count_1"]], axis=0)
grouped_df["Count_-1 Ratio"] = grouped_df["Count_-1"] / total_counts
grouped_df["Count_0 Ratio"] = grouped_df["Count_0"] / total_counts
grouped_df["Count_1 Ratio"] = grouped_df["Count_1"] / total_counts
grouped_df

,Date,sentiment_numeric_finbert,sentiment_score_finbert,Close,Count_-1,Count_0,Count_1,Count_-1 Ratio,Count_0 Ratio,Count_1 Ratio
0,2017-12-18,-0.133333,-0.123620,2690.159912,16,36,8,0.266667,0.600000,0.133333
1,2017-12-19,-0.150000,-0.141546,2681.469971,4,15,1,0.200000,0.750000,0.050000
2,2017-12-20,-0.100000,-0.091164,2679.250000,3,16,1,0.150000,0.800000,0.050000
3,2017-12-21,-0.050000,-0.045027,2684.570068,3,15,2,0.150000,0.750000,0.100000
4,2017-12-22,-0.086957,-0.059859,2683.340088,4,17,2,0.173913,0.739130,0.086957
...,...,...,...,...,...,...,...,...,...,...
642,2020-07-13,-0.016949,-0.001700,3155.219971,22,76,20,0.186441,0.644068,0.169492
643,2020-07-14,-0.045045,-0.049339,3197.520020,25,66,20,0.225225,0.594595,0.180180
644,2020-07-15,0.025210,0.014326,3226.560059,22,72,25,0.184874,0.605042,0.210084
645,2020-07-16,-0.176991,-0.161201,3215.570068,30,73,10,0.265487,0.646018,0.088496


In [19]:
df_final = grouped_df.copy()
df_final['Price_Change'] = df_final['Close'].pct_change()
df_final['Price_Label'] = df_final['Price_Change'].apply(lambda x: -1 if x < 0 else 1 if x>0 else 0)
df_final.reset_index(inplace=True,drop=True)
df_final

,Date,sentiment_numeric_finbert,sentiment_score_finbert,Close,Count_-1,Count_0,Count_1,Count_-1 Ratio,Count_0 Ratio,Count_1 Ratio,Price_Change,Price_Label
0,2017-12-18,-0.133333,-0.123620,2690.159912,16,36,8,0.266667,0.600000,0.133333,NaN,0
1,2017-12-19,-0.150000,-0.141546,2681.469971,4,15,1,0.200000,0.750000,0.050000,-0.003230,-1
2,2017-12-20,-0.100000,-0.091164,2679.250000,3,16,1,0.150000,0.800000,0.050000,-0.000828,-1
3,2017-12-21,-0.050000,-0.045027,2684.570068,3,15,2,0.150000,0.750000,0.100000,0.001986,1
4,2017-12-22,-0.086957,-0.059859,2683.340088,4,17,2,0.173913,0.739130,0.086957,-0.000458,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
642,2020-07-13,-0.016949,-0.001700,3155.219971,22,76,20,0.186441,0.644068,0.169492,-0.009363,-1
643,2020-07-14,-0.045045,-0.049339,3197.520020,25,66,20,0.225225,0.594595,0.180180,0.013406,1
644,2020-07-15,0.025210,0.014326,3226.560059,22,72,25,0.184874,0.605042,0.210084,0.009082,1
645,2020-07-16,-0.176991,-0.161201,3215.570068,30,73,10,0.265487,0.646018,0.088496,-0.003406,-1


In [20]:
df_ml = df_final.loc[:,['Date', 'sentiment_numeric_finbert', 'sentiment_score_finbert','Count_-1 Ratio', 'Count_0 Ratio','Count_1 Ratio', 'Close','Price_Change', 'Price_Label']]

### Using RandomForest, LogisticRegression and Gradint Boosting on Scores and Count Data

In [23]:
# Feature matrix and target
X = df_ml[['sentiment_score_finbert',
           'sentiment_numeric_finbert', 'Count_-1 Ratio', 'Count_0 Ratio', 'Count_1 Ratio']]
y = df_final['Price_Label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    "Random Forest": RandomForestClassifier(random_state=42, class_weight="balanced"),
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Evaluate each model
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    rec = recall_score(y_test, y_pred, average="weighted")
    f1 = f1_score(y_test, y_pred, average="weighted")

    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1
    })

# Create and display results dataframe
results_df = pd.DataFrame(results)
results_df

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1 Score
0,Random Forest,0.576923,0.574636,0.576923,0.569906
1,Logistic Regression,0.476923,0.594406,0.476923,0.518167
2,Gradient Boosting,0.553846,0.550404,0.553846,0.547357


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
feature_importances = clf.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
print(feature_importance_df)
plt.figure(figsize=(8, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], align='center')
plt.xlabel("Feature Importance")
plt.ylabel("Features")
plt.title("Feature Importance in Random Forest")
plt.gca().invert_yaxis()
plt.show()

### Using ML Models with lagged data(n=5)

In [24]:
max_lag = 5
base_features = ['sentiment_score_finbert', 'sentiment_numeric_finbert',
                 'Count_-1 Ratio', 'Count_0 Ratio', 'Count_1 Ratio']

# Add lagged features
for lag in range(1, max_lag + 1):
    for col in base_features:
        df_ml[f'{col}_lag{lag}'] = df_ml[col].shift(lag)

# Drop rows with NaN from lagging
df_ml.dropna(inplace=True)

# Define full feature set
features = base_features.copy()
for lag in range(1, max_lag + 1):
    features.extend([f'{col}_lag{lag}' for col in base_features])

# Prepare X and y
X = df_ml[features]
y = df_ml['Price_Label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Models to evaluate
models = {
    "Random Forest": RandomForestClassifier(random_state=42, class_weight="balanced"),
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Evaluation loop
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
        "Recall": recall_score(y_test, y_pred, average='weighted'),
        "F1 Score": f1_score(y_test, y_pred, average='weighted')
    })

# Display results
results_df = pd.DataFrame(results)
results_df

,Model,Accuracy,Precision,Recall,F1 Score
0,Random Forest,0.620155,0.620524,0.620155,0.611643
1,Logistic Regression,0.643411,0.646968,0.643411,0.643797
2,Gradient Boosting,0.503876,0.497129,0.503876,0.496158


### ML Models with lagged Data and Fundamental Indicators

In [32]:
merged_data = df_ml.copy()
close_price = merged_data['Close'].shift(1)
merged_data['RSI'] = ta.momentum.RSIIndicator(close=close_price, window=14).rsi()
merged_data['EMA'] = ta.trend.EMAIndicator(close=close_price, window=14).ema_indicator()
macd = ta.trend.MACD(close=close_price)
merged_data['MACD'] = macd.macd()
merged_data['MACD_Signal'] = macd.macd_signal()
bb = ta.volatility.BollingerBands(close=close_price)
merged_data['BB_Upper'] = bb.bollinger_hband()
merged_data['BB_Lower'] = bb.bollinger_lband()
merged_data['SMA'] = ta.trend.SMAIndicator(close=close_price, window=14).sma_indicator()


trend_map = {
    'RSI_Trend': 'RSI',
    'EMA_Trend': 'EMA',
    'MACD_Trend': 'MACD',
    'BB_Trend': 'BB_Upper',
    'SMA_Trend': 'SMA'
}
for new_col, base_col in trend_map.items():
    merged_data[new_col] = merged_data[base_col].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))


merged_data.dropna(inplace=True)
merged_data.reset_index(drop=True, inplace=True)


base_sentiment = ['sentiment_numeric_finbert', 'sentiment_score_finbert',
                  'Count_-1 Ratio', 'Count_0 Ratio', 'Count_1 Ratio']
lagged_sentiment = [f"{col}_lag{lag}" for lag in range(1, 6) for col in base_sentiment]
trend_features = list(trend_map.keys())
all_features = base_sentiment + lagged_sentiment + trend_features


X = merged_data[all_features]
y = merged_data['Price_Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


models = {
    "Random Forest": RandomForestClassifier(random_state=42, class_weight="balanced"),
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}


results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
        "Recall": recall_score(y_test, y_pred, average='weighted'),
        "F1 Score": f1_score(y_test, y_pred, average='weighted')
    })

results_df = pd.DataFrame(results)
results_df

,Model,Accuracy,Precision,Recall,F1 Score
0,Random Forest,0.532787,0.524958,0.532787,0.524552
1,Logistic Regression,0.647541,0.655888,0.647541,0.648181
2,Gradient Boosting,0.557377,0.553500,0.557377,0.554094


### ML Models with PCA and RFE, GridSearchCV

In [53]:
sentiment_base = ['sentiment_score_finbert', 'sentiment_numeric_finbert']
fundamental_base = ['Count_-1 Ratio', 'Count_0 Ratio', 'Count_1 Ratio']
base_features = sentiment_base + fundamental_base
max_lag = 5
for lag in range(1, max_lag + 1):
    for col in base_features:
        merged_data[f"{col}_lag{lag}"] = merged_data[col].shift(lag)

# Drop missing
merged_data.dropna(inplace=True)
merged_data.reset_index(drop=True, inplace=True)

# Feature list
trend_features = list(trend_map.keys())
lagged_features = [f"{col}_lag{lag}" for lag in range(1, max_lag + 1) for col in base_features]
all_features = base_features + lagged_features + trend_features
X = merged_data[all_features]
y = merged_data['Price_Label']

# Feature transformations
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_scaled)

rfe = RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=10)
X_rfe = rfe.fit_transform(X_scaled, y)

# Splits
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=42)
X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe = train_test_split(X_rfe, y, test_size=0.2, random_state=42)

# Parameter grids
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1'],
    'solver': ['liblinear', 'saga']
}
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
param_grid_gb = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0]
}

# Models and datasets
model_configs = [
    ("Logistic Regression", LogisticRegression(max_iter=1000, class_weight="balanced"), param_grid_lr),
    ("Random Forest", RandomForestClassifier(random_state=42, class_weight="balanced"), param_grid_rf),
    ("Gradient Boosting", GradientBoostingClassifier(random_state=42), param_grid_gb)
]
datasets = [
    ("Raw", X_train, X_test, y_train, y_test),
    ("PCA", X_train_pca, X_test_pca, y_train_pca, y_test_pca),
    ("RFE", X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe)
]

# Evaluation
def evaluate_model(name, variant, model, X_te, y_te):
    y_pred = model.predict(X_te)
    return {
        "Model": name,
        "Feature Variant": variant,
        "Accuracy": accuracy_score(y_te, y_pred),
        "Precision": precision_score(y_te, y_pred, average='weighted', zero_division=0),
        "Recall": recall_score(y_te, y_pred, average='weighted'),
        "F1 Score": f1_score(y_te, y_pred, average='weighted')
    }

# GridSearch and collect results
results = []
for model_name, base_model, param_grid in model_configs:
    for variant, X_tr, X_te, y_tr, y_te in datasets:
        grid = GridSearchCV(base_model, param_grid, scoring='f1_macro', cv=5)
        grid.fit(X_tr, y_tr)
        best_model = grid.best_estimator_
        results.append(evaluate_model(model_name, variant, best_model, X_te, y_te))

# Display all results
results_df = pd.DataFrame(results)

In [55]:
results_df.sort_values(by='F1 Score',ascending=False)

,Model,Feature Variant,Accuracy,Precision,Recall,F1 Score
7,Gradient Boosting,PCA,0.654867,0.654134,0.654867,0.648069
0,Logistic Regression,Raw,0.637168,0.635043,0.637168,0.634848
4,Random Forest,PCA,0.637168,0.634871,0.637168,0.631888
2,Logistic Regression,RFE,0.628319,0.626344,0.628319,0.626541
1,Logistic Regression,PCA,0.610619,0.609395,0.610619,0.609816
8,Gradient Boosting,RFE,0.619469,0.629348,0.619469,0.589618
3,Random Forest,Raw,0.584071,0.578516,0.584071,0.570657
6,Gradient Boosting,Raw,0.584071,0.579011,0.584071,0.564096
5,Random Forest,RFE,0.557522,0.552277,0.557522,0.552102


In [56]:
best_row = results_df.sort_values(by='F1 Score', ascending=False).iloc[0]
best_model_name = best_row['Model']
best_feature_variant = best_row['Feature Variant']

variant_to_data = {
    "Raw": (X_train, X_test, y_train, y_test, X.columns),
    "PCA": (X_train_pca, X_test_pca, y_train_pca, y_test_pca, [f'PC{i+1}' for i in range(X_pca.shape[1])]),
    "RFE": (X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe, list(X.columns[rfe.get_support()]))
}
X_tr_best, X_te_best, y_tr_best, y_te_best, feature_names = variant_to_data[best_feature_variant]

grid = None
if best_model_name == "Logistic Regression":
    base_model = LogisticRegression(max_iter=1000, class_weight="balanced")
    param_grid = param_grid_lr
elif best_model_name == "Random Forest":
    base_model = RandomForestClassifier(random_state=42, class_weight="balanced")
    param_grid = param_grid_rf
elif best_model_name == "Gradient Boosting":
    base_model = GradientBoostingClassifier(random_state=42)
    param_grid = param_grid_gb

# Refit best model
grid = GridSearchCV(base_model, param_grid, scoring='f1_macro', cv=5)
grid.fit(X_tr_best, y_tr_best)
best_model = grid.best_estimator_

if best_model_name == "Logistic Regression":
    importance = best_model.coef_[0]
elif best_model_name in ["Random Forest", "Gradient Boosting"]:
    importance = best_model.feature_importances_

importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importance
}).sort_values(by="Importance", ascending=False).head(10)
pca_components = pca.components_  


original_feature_names = X.columns

top_pc_names = importance_df['Feature'].values
top_pc_indices = [int(pc[2:]) - 1 for pc in top_pc_names]  

contributions = []

for pc_idx in top_pc_indices:
    pc_name = f"PC{pc_idx+1}"
    component_weights = pca_components[pc_idx]
    pc_contrib_df = pd.DataFrame({
        'Original Feature': original_feature_names,
        'Contribution Weight': component_weights,
        'Principal Component': pc_name
    }).sort_values(by='Contribution Weight', key=abs, ascending=False).head(10)
    contributions.append(pc_contrib_df)

top_pc_contributions_df = pd.concat(contributions, ignore_index=True)

In [59]:
top_pc_contributions_df.head(2)

,Original Feature,Contribution Weight,Principal Component
0,sentiment_score_finbert,0.342931,PC7
1,sentiment_numeric_finbert,0.341969,PC7
